In [ ]:
# !pip3 install openpyxl

In [1]:
import requests
import base64
import os
import numpy as np
import pandas as pd
import json
import time

In [162]:
class Spotify():
    '''
    Class to handle hitting spotify API to get music features
    '''
    
    def __init__(self, 
                 client_id=None, 
                 client_secret=None):
        
        self.client_id = client_id
        self.client_secret = client_secret
        
        self.get_access_token()
        
        
    def get_access_token(self):
        ## Example use:
        ## spotify_instance = Spotify(CLIENT_ID, CLIENT_SECRET)
        ## spotify_instance.get_access_token()
        
        secret_bytes = bytes(('{}:{}'.format(self.client_id, self.client_secret)),'utf-8')
        secret_enc = base64.b64encode(secret_bytes).decode('utf-8')
        
        data = {'grant_type': 'client_credentials'}
        headers = {'Authorization': 'Basic {}'.format(secret_enc)}
        url = 'https://accounts.spotify.com/api/token'
        r = requests.post(url, headers=headers, data=data)
        
        self.access_token = r.json()['access_token']
    
    def lookup_spotify_id(self, id_list):
        '''
        this takes in ISRC and returns ISRC to Spotify ID map
        '''
        df = pd.DataFrame([])
        bad_ids = []
        for i in id_list:
            r = requests.get(f'https://api.spotify.com/v1/search?type=track&q=isrc:{i}',
                             headers = {'Authorization': 'Bearer ' + self.access_token})
            while r.status_code == 429:
                retry_secs = int(r.headers['Retry-After'])
                print('sleeping for {}'.format(retry_secs))
                time.sleep(retry_secs)
            try:
                spotify_id = r.json()['tracks']['items'][0]['id']
                row = {'isrc': str(i), 'spotify_id': spotify_id}
                df = df.append(row, ignore_index = True)
            except:
#                 print(f'id issue {len(bad_ids) + 1}')
                row = {'isrc': str(i), 'spotify_id': None}
                df = df.append(row, ignore_index = True)
                bad_ids.append(i)
        return df
    
    def lookup_album_id(self, id_list):
        '''
        this takes in UPC and returns UPC to Spotify Album ID
        '''
        df = pd.DataFrame([])
        bad_ids = []
        for i in id_list:
            r = requests.get(f'https://api.spotify.com/v1/search?type=album&q=upc:{i}',
                             headers = {'Authorization': 'Bearer ' + self.access_token})
            while r.status_code == 429:
                retry_secs = int(r.headers['Retry-After'])
                print('sleeping for {}'.format(retry_secs))
                time.sleep(retry_secs)
            try:
                spotify_id = r.json()['albums']['items'][0]['id']
                row = {'upc': str(i), 'spotify_album_id': spotify_id}
                df = df.append(row, ignore_index = True)
            except:
#                 print(f'id issue {len(bad_ids) + 1}')
                row = {'upc': str(i), 'spotify_album_id': None}
                df = df.append(row, ignore_index = True)
                bad_ids.append(i)
        return df
    
    def group_ids(self, list_of_ids, bucket_size=100):
        list_of_ids = pd.DataFrame(list_of_ids, columns=['id'])
        list_of_ids['bucket'] = np.floor(np.arange(len(list_of_ids)) / bucket_size)
        
        gpd_ids = list_of_ids.groupby('bucket')['id'].apply(lambda x: ','.join(x))
        
        return gpd_ids

    def get_audio_features(self, id_list, bucket_size=100):
        
        gpd_ids = self.group_ids(id_list, bucket_size)
        col_list = ['acousticness', 'liveness', 'instrumentalness', 
                    'analysis_url', 'uri', 'time_signature', 'loudness', 
                    'speechiness', 'duration_ms', 'danceability', 'mode', 
                    'id', 'energy', 'key', 'track_href', 
                    'valence', 'type', 'tempo']
        
        df = []
        
        for x in gpd_ids.values:
            r = requests.get('https://api.spotify.com/v1/audio-features?ids={}'.format(x), 
                             headers = {'Authorization': 'Bearer ' + self.access_token})
            while r.status_code == 429:
                retry_secs = int(r.headers['Retry-After'])
                time.sleep(retry_secs)
            
            audio_features = r.json()['audio_features']
            audio_features = [i for i in audio_features if i]
            
            for row in audio_features:
                for c in col_list:
                    if c not in row:
                        row[c] = None
                df.append(row)
    
        return pd.DataFrame(df)
    
    def get_tracks(self, id_list, bucket_size=50):
        
        gpd_ids = self.group_ids(id_list, bucket_size)
        col_list = ['popularity', 'name', 'uri', 'external_urls', 'type', 'duration_ms', 'external_ids',
                    'album', 'explicit', 'id', 'preview_url', 'track_number', 'available_markets', 'is_local', 
                    'artists', 'href', 'disc_number']
        
        track_df = []
        
        for x in gpd_ids.values:
            r = requests.get('https://api.spotify.com/v1/tracks?ids={}'.format(x),
                             headers = {'Authorization': 'Bearer ' + self.access_token})
            
            while r.status_code == 429:
                retry_secs = int(r.headers['Retry-After'])
                time.sleep(retry_secs)
                  
            tracks = r.json()['tracks']
            tracks = [i for i in tracks if i]
            
            for row in tracks:
                for c in col_list:
                    if c not in row:
                        row[c] = None
                track_df.append(row)
        
        final_track_df = pd.DataFrame(track_df)
        
        final_track_df['release_date'] = pd.json_normalize(final_track_df['album'])['release_date']
        final_track_df['album_id'] = pd.json_normalize(final_track_df['album'])['id']
        
        return final_track_df[['id','popularity','album_id','release_date']]

    def get_albums(self, id_list, bucket_size=20):
        
        gpd_ids = self.group_ids(id_list, bucket_size)
        col_list = ['id','release_date', 'release_date_precision','total_tracks','type']
        
        album_df = []
        
        for x in gpd_ids.values:
            r = requests.get('https://api.spotify.com/v1/albums?ids={}'.format(x),
                             headers = {'Authorization': 'Bearer ' + self.access_token})
            
            while r.status_code == 429:
                retry_secs = int(r.headers['Retry-After'])
                time.sleep(retry_secs)
                  
            albums = r.json()['albums']
            albums = [i for i in albums if i]
            
            for a in albums:
                df_row = {x: a[x] for x in col_list}
                album_df.append(df_row)
                
        final_album_df = pd.DataFrame(album_df)
        return final_album_df

In [163]:
client_id = 'ae17fb14354d4d98a442007563fafab9'
client_secret = 'cb9e3752f07f4cfc9685c9369fd4b11c'

sp = Spotify(client_id,client_secret)
sp.get_access_token()

In [175]:
data_path = r'data/'
dataset_path = r'data/mri_full_data.csv'

music_data = pd.read_csv(dataset_path)
music_data.drop('Unnamed: 0', axis=1, inplace=True)
music_data['Display Upc'] = music_data['Display Upc'].astype('string')

In [165]:
isrc_ids = list(music_data[~music_data.ISRC.isna()].ISRC.unique())
upc_ids = list(music_data[~music_data['Display Upc'].isna()]['Display Upc'].unique())

In [166]:
isrc_to_spotify_id = sp.lookup_spotify_id(isrc_ids)

In [167]:
upc_to_spotify_album_id = sp.lookup_album_id(upc_ids)

In [168]:
isrc_to_spotify_id.to_csv(os.path.join(data_path, 'isrc_to_spotify_id.csv'))
upc_to_spotify_album_id.to_csv(os.path.join(data_path, 'upc_to_spotify_album_id.csv'))

In [169]:
spotify_ids = (isrc_to_spotify_id[~(isrc_to_spotify_id.spotify_id.isna())].spotify_id.unique())

In [170]:
track_audio_feats = sp.get_audio_features(spotify_ids)

In [171]:
track_audio_feats.to_csv(os.path.join(data_path, 'track_audio_feats.csv'))

In [172]:
spotify_album_ids = list(
    upc_to_spotify_album_id[
        ~(upc_to_spotify_album_id.spotify_album_id.isna())
    ].spotify_album_id.unique()
)

In [173]:
album_information = sp.get_albums(spotify_album_ids)

In [174]:
album_information.to_csv(os.path.join(data_path, 'album_information.csv'))

# join & write all data

In [182]:
music_data_merged_spotify = music_data.merge(
    isrc_to_spotify_id,
    how = 'left',
    left_on = 'ISRC',
    right_on = 'isrc'
).merge(
    upc_to_spotify_album_id,
    how = 'left',
    left_on = 'Display Upc',
    right_on = 'upc'
).merge(
    track_audio_feats,
    how = 'left',
    left_on = 'spotify_id',
    right_on = 'id'
).drop(
    ['id', 'uri','track_href','analysis_url','type'],
    axis = 1
).merge(
    album_information,
    how = 'left',
    left_on = 'spotify_album_id',
    right_on = 'id'
).drop(
    ['id', 'isrc', 'upc'],
    axis = 1
)

In [191]:
music_data_merged_spotify['type'] = np.where(
    (music_data_merged_spotify.type.isna()) & ~(music_data_merged_spotify.spotify_id.isna()),
    'track',
    music_data_merged_spotify.type
)

In [192]:
music_data_merged_spotify.to_csv(os.path.join(data_path, 'music_data_merged_spotify.csv'))

# joining stats

In [205]:
len_match = len(music_data_merged_spotify[~(music_data_merged_spotify.type.isna())])
len_df = len(music_data_merged_spotify)
print (f'{len_match} out of {len_df} rows have spotify information ({np.round((len_match/len_df) * 100, 1)}%)')

4704716 out of 4757930 rows have spotify information (98.9%)
